# Перованачальное скачивание данных

In [ ]:
!pip install pytube

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 5.7 MB/s eta 0:00:00


In [ ]:
import os
import numpy as np
import pandas as pd
import subprocess as sp

from tqdm import tqdm
from pytube import YouTube
from torchvision.datasets.kinetics import Kinetics

Скачаем и посмотрим csv файлы для train и valid выборок
(Пример для одного файла)

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/kinetics700_2020/train.csv')
data

,label,youtube_id,time_start,time_end,split
0,clay pottery making,---0dWlqevI,19,29,train
1,news anchoring,---aQ-tA5_A,9,19,train
2,using bagging machine,---j12rm3WI,14,24,train
3,javelin throw,--07WQ2iBlw,1,11,train
4,climbing a rope,--0NTAs-fA0,29,39,train
...,...,...,...,...,...
532901,washing dishes,zzz_3yWpTXo,0,10,train
532902,juggling fire,zzzkS3amkWE,124,134,train
532903,taking photo,zzzsd1R7H0E,6,16,train
532904,brush painting,zzzxltuPx2Q,84,94,train


Выберем только те видео, в названии класса которых есть слово 'dancing'

In [ ]:
dancing_data = data[data['label'].str.contains(r"\bdancing\b", case = False)]
dancing_data

,label,youtube_id,time_start,time_end,split
17,tap dancing,--6q_33gNew,132,142,train
166,belly dancing,-0L4z8D2eGs,43,53,train
179,tap dancing,-0SUZKVInwE,81,91,train
298,dancing charleston,-1KtlEOx78I,9,19,train
299,dancing ballet,-1Kv095GbV8,0,10,train
...,...,...,...,...,...
532751,tango dancing,zyN5s4RTthw,175,185,train
532759,tango dancing,zyOZVfNrbX0,120,130,train
532760,dancing ballet,zyOb-PosWSY,68,78,train
532772,tango dancing,zyY6SLwKvrk,1,11,train


Так как видео достаточно много, выберем по 100 случайных объектов каждого класса, для ускорения эксперимента

In [ ]:
data_short = dancing_data.groupby('label', as_index=False).apply(lambda array: array.loc[np.random.choice(array.index, 100, False),:])
data_short

label   youtube_id  time_start  time_end  split
0  382496  belly dancing  iUIf-_ZP1UY         169       179  train
   264110  belly dancing  V7YC5aXsMUY          39        49  train
   162251  belly dancing  JU1sgKMVIlg          61        71  train
   105199  belly dancing  Cev08b5Nhg0          57        67  train
   146333  belly dancing  HcWWsrezVlk          47        57  train
...                  ...          ...         ...       ...    ...
13 397559    tap dancing  kDqZPSLFqJw          46        56  train
   406466    tap dancing  lFN-7WrfZks          15        25  train
   19969     tap dancing  1g6vpPU8Z2k          16        26  train
   17639     tap dancing  1MPJSccFb2c           0        10  train
   500353    tap dancing  w4iSDvhKhkg          38        48  train

[1400 rows x 5 columns]

In [ ]:
count_failed = 0

for index, row in tqdm(data_short.iterrows()):
  try:
    url = YouTube('http://youtube.com/watch?v={}'.format(str(row['youtube_id']))).streams.filter(progressive=True, file_extension='mp4').order_by('resolution').desc().first().url
    ss = row['time_start']
    t = 10
    download_file_path = "my_" + str(row['youtube_id']) + '.mp4'
    process_call_str = 'ffmpeg -ss {1} -t {2} -i "{0}" -an "{3}" -y'.format(str(url), str(ss), str(t), download_file_path)
    status = sp.check_call(process_call_str, shell=True)
    print("success on: " + str(row['youtube_id']))
  except Exception as e:
    count_failed += 1
    print(e)

print(count_failed)

В результате скачивания получилось ~1000 train объектов и ~130 valid объектов (из-за проблем с доступом к некоторым видео получилось скачать не все)

In [ ]:
#скрипты, использовавшиеся для создания необходимой структуры данных
labels = dancing_data['label'].unique()

for label in labels:
  print(label)
  if not os.path.exists('val/' + label):
      os.makedirs('val/' + label)

In [ ]:
from shutil import move

for filename in os.listdir(os.getcwd()):
  if filename.endswith("mp4"): 
    class_name = list(dancing_data.loc[dancing_data['youtube_id'] == filename[3:-4]]['label'])[0]
    move(filename, "/content/val/" + class_name + '/' + filename)

In [ ]:
!zip -r /content/val.zip /content/val

In [ ]:
from google.colab import files
files.download('val.zip') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>